# Create ISM supplemental data 
- ISM results for all positions.

In [8]:
import numpy as np
import gzip
bases = ['A', 'C', 'G', 'T']

# Load data

In [2]:
from genome import Genome
genome = Genome('/Users/jpaggi/Downloads/MPRA/snps/hg19.fa')

In [3]:
# file is result of bedtools intersect -loj
introns = {}
with open('../preprocessing/introns_to_mercer.tsv') as fp:
    for line in fp:
        chrom, start, end, _, pos, strand, _, bp = line.split('\t')[:8]
        bp, start, end = int(bp), int(start), int(end)
        
        three = end if strand == '+' else start  
        key = (chrom, three, strand)
        
        if not 5 < abs(bp - three) < 60:
            bp = -1
        
        if key not in introns: introns[key] = []
        if (bp not in introns[key]
            and bp != -1): introns[key] += [bp]

In [4]:
L = 70

def onehot(seq):
    X = np.zeros((len(seq), len(bases)))
    for i, char in enumerate(seq):
        X[i, bases.index(char)] = 1
    return X

def encode(introns):
    X, y, chroms, coords, strands = [], [], [], [], []
    for intron, bps in introns.items():
        chrom, three, strand = intron
        if strand == '+':
            begin, stop = three - L, three
        else:
            begin, stop = three, three + L

        # Get features
        seq = genome.get_seq(chrom, begin, stop, strand)
        if 'N' in seq: seq = seq.replace('N', 'A')

        X += [onehot(seq).reshape(1, 70, 4)]

        # Make target
        _y = np.zeros((stop - begin,))
        for bp in bps:
            if strand == '+':
                bp = L + bp - three
            else:
                bp = L - bp + three - 1
            _y[bp] = 1
        y += [_y]
        
        chroms += [chrom]
        coords += [three]
        strands += [strand]
    return (np.vstack(X), np.vstack(y),
            np.array(chroms), np.array(coords), np.array(strands))

(X, y, chroms, coords, strands) = encode(introns)

In [7]:
from keras.models import load_model
model = load_model('../models/2layer.h5')

# Write genome-wide ISM to file

In [51]:
# Should take about 10 hours...
done = 0
with gzip.open('../anno/lstm.ism.tsv.gz', 'w') as fp:
    fp.write('\t'.join(['chrom', 'pos', 'ref', 'alt', 'strand', '3pSS', 'd_best_bp', 'd_max_bp'])+'\n')
    for p, seq, chrom, pos, strand in zip(preds, X, chroms, coords, strands):
        # Construct alternative sequences
        seqs = [] 
        for i in range(70):
            for j in range(4):
                if seq[i, j]: continue
                s = np.zeros(seq.shape)
                s[seq == 1] = 1
                s[i] = 0
                s[i, j] = 1
                seqs += [s]
        p = p.flatten()

        altpreds = model.predict(np.vstack(seqs).reshape(-1, 70, 4)).reshape(-1, 70)

        maxdelta  = np.max(altpreds, axis = 1) - np.max(p)
        bestdelta = altpreds[:, np.argmax(p)] - np.max(p)

        idx = 0
        for i in range(70):
            coord = pos + (i-L+1 if strand == '+' else L-i)
            ref = bases[np.argmax(seq[i])]
            if strand == '-': ref = genome.revcomp(ref)
            for j, alt in enumerate(bases):
                if seq[i, j]: continue
                if strand == '-': alt = genome.revcomp(alt)
                fp.write('\t'.join(map(str, [chrom, coord, ref, alt, strand, pos,
                                             bestdelta[i], maxdelta[i]]))+'\n')
                idx += 1
        if not done % 1000: print done
        done += 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
